In [2]:
PYTHONPATH=. python tools/test.py \
  configs/segnext/segnext_mscan-l_1xb16-adamw-40k_plantseg115-512x512.py \
  work_dirs/segnext_mscan-l_1xb16-adamw-40k_plantseg115-512x512/iter_20000.pth \
  --work-dir work_dirs/segnext_mscan-l_1xb16-adamw-40k_plantseg115-512x512 \
  --show-dir work_dirs/segnext_mscan-l_1xb16-adamw-40k_plantseg115-512x512/test_vis \
  --cfg-options val_evaluator.type=IoUMetric val_evaluator.iou_metrics='["mIoU"]'


SyntaxError: invalid decimal literal (144934965.py, line 2)

In [4]:
# Set PYTHONPATH if needed
%env PYTHONPATH=.

# Run model evaluation and compute mIoU on the validation set
!python tools/test.py \
  configs/segnext/segnext_mscan-l_1xb16-adamw-40k_plantseg115-512x512.py \
  work_dirs/segnext_mscan-l_1xb16-adamw-40k_plantseg115-512x512/iter_8000.pth \
  --cfg-options val_evaluator.type=IoUMetric val_evaluator.iou_metrics="['mIoU']" \
  --work-dir work_dirs/segnext_mscan-l_1xb16-adamw-40k_plantseg115-512x512


env: PYTHONPATH=.
/bin/sh: 1: gcc: not found
05/14 08:45:10 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 750847637
    GPU 0: Quadro RTX 6000
    CUDA_HOME: None
    GCC: n/a
    PyTorch: 2.1.0
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_61,code=sm_61;-

In [3]:
import os

path = 'work_dirs/segnext_mscan-l_1xb16-adamw-40k_plantseg115-512x512/iter_8000.pth'
size_mb = os.path.getsize(path) / (1024 * 1024)
print(f"Checkpoint size: {size_mb:.2f} MB")


Checkpoint size: 562.50 MB


In [16]:
import os
import time
import torch
import numpy as np
import logging
from mmseg.apis import init_model
from mmengine.config import Config
from mmseg.registry import RUNNERS

# ✂️ Suppress excessive logging
logging.getLogger('mmengine').setLevel(logging.ERROR)

# ✅ Path configuration
config_path = 'configs/segnext/segnext_mscan-l_1xb16-adamw-40k_plantseg115-512x512.py'
checkpoint_path = 'work_dirs/segnext_mscan-l_1xb16-adamw-40k_plantseg115-512x512/iter_8000.pth'

# ✅ Load config and model
cfg = Config.fromfile(config_path)
cfg.log_level = 'ERROR'
cfg.work_dir = 'work_dirs/segnext_mscan-l_1xb16-adamw-40k_plantseg115-512x512'
model = init_model(cfg, checkpoint_path, device='cuda')
model.eval()

# ✅ Build runner and obtain the test_loader (includes inputs and data_samples)
runner = RUNNERS.build(cfg)
test_loader = runner.test_dataloader

# ✅ Fetch one batch and move it to CUDA
batch_data = next(iter(test_loader))
batch_data = {
    k: [x.to('cuda') for x in v] if isinstance(v, list) else v
    for k, v in batch_data.items()
}

# ✅ Warm-up pass
with torch.no_grad():
    _ = model.test_step(batch_data)

# ✅ Latency test: multiple forward passes on a single batch
num_trials = 100
latencies = []
with torch.no_grad():
    for _ in range(num_trials):
        start = time.time()
        _ = model.test_step(batch_data)
        latencies.append(time.time() - start)

# ✅ Throughput test: measure processing speed over multiple batches
num_batches = 50
batch_times = []
with torch.no_grad():
    for _ in range(num_batches):
        start = time.time()
        _ = model.test_step(batch_data)
        batch_times.append(time.time() - start)

# ✅ Calculate model file size
model_size = os.path.getsize(checkpoint_path)

# ✅ Print performance report
print("\n📊 === SegNeXt Serving Performance Report ===")
print(f"💾 Model Size on Disk: {model_size / (1024 ** 2):.2f} MB")
print(f"⏱️ Inference Latency (median): {np.percentile(latencies, 50) * 1000:.2f} ms")
print(f"⏱️ Inference Latency (95th percentile): {np.percentile(latencies, 95) * 1000:.2f} ms")
print(f"⏱️ Inference Latency (99th percentile): {np.percentile(latencies, 99) * 1000:.2f} ms")
print(f"🚀 Inference Throughput (single sample): {num_trials / np.sum(latencies):.2f} FPS")
print(f"🚀 Batch Throughput: {len(batch_data['inputs']) * num_batches / np.sum(batch_times):.2f} FPS")


Loads checkpoint by local backend from path: work_dirs/segnext_mscan-l_1xb16-adamw-40k_plantseg115-512x512/iter_8000.pth


/bin/sh: 1: gcc: not found



📊 === SegNeXt Serving Performance Report ===
💾 Model Size on Disk: 562.50 MB
⏱️ Inference Latency (median): 41.89 ms
⏱️ Inference Latency (95th percentile): 42.33 ms
⏱️ Inference Latency (99th percentile): 50.06 ms
🚀 Inference Throughput (single sample): 23.68 FPS
🚀 Batch Throughput: 23.83 FPS


In [17]:
import os
import time
import torch
import numpy as np
import logging
from mmseg.apis import init_model
from mmengine.config import Config
from mmseg.registry import RUNNERS

# ✂️ Suppress excessive logging
logging.getLogger('mmengine').setLevel(logging.ERROR)

# ✅ Path configuration
config_path = 'configs/segnext/segnext_mscan-l_1xb16-adamw-40k_plantseg115-512x512.py'
checkpoint_path = 'work_dirs/segnext_mscan-l_1xb16-adamw-40k_plantseg115-512x512/iter_8000.pth'

# ✅ Load config and model
cfg = Config.fromfile(config_path)
cfg.log_level = 'ERROR'
cfg.work_dir = 'work_dirs/segnext_mscan-l_1xb16-adamw-40k_plantseg115-512x512'
model = init_model(cfg, checkpoint_path, device='cpu')
model.eval()

# ✅ Build runner and obtain the test_loader (includes inputs and data_samples)
runner = RUNNERS.build(cfg)
test_loader = runner.test_dataloader

# ✅ Fetch one batch and move it to CPU
batch_data = next(iter(test_loader))
batch_data = {
    k: [x.to('cpu') for x in v] if isinstance(v, list) else v
    for k, v in batch_data.items()
}

# ✅ Warm-up pass
with torch.no_grad():
    _ = model.test_step(batch_data)

# ✅ Latency test: multiple forward passes on a single batch
num_trials = 100
latencies = []
with torch.no_grad():
    for _ in range(num_trials):
        start = time.time()
        _ = model.test_step(batch_data)
        latencies.append(time.time() - start)

# ✅ Throughput test: measure processing speed over multiple batches
num_batches = 50
batch_times = []
with torch.no_grad():
    for _ in range(num_batches):
        start = time.time()
        _ = model.test_step(batch_data)
        batch_times.append(time.time() - start)

# ✅ Calculate model file size
model_size = os.path.getsize(checkpoint_path)

# ✅ Print performance report
print("\n📊 === SegNeXt Serving Performance Report ===")
print(f"💾 Model Size on Disk: {model_size / (1024 ** 2):.2f} MB")
print(f"⏱️ Inference Latency (median): {np.percentile(latencies, 50) * 1000:.2f} ms")
print(f"⏱️ Inference Latency (95th percentile): {np.percentile(latencies, 95) * 1000:.2f} ms")
print(f"⏱️ Inference Latency (99th percentile): {np.percentile(latencies, 99) * 1000:.2f} ms")
print(f"🚀 Inference Throughput (single sample): {num_trials / np.sum(latencies):.2f} FPS")
print(f"🚀 Batch Throughput: {len(batch_data['inputs']) * num_batches / np.sum(batch_times):.2f} FPS")


Loads checkpoint by local backend from path: work_dirs/segnext_mscan-l_1xb16-adamw-40k_plantseg115-512x512/iter_8000.pth


/bin/sh: 1: gcc: not found



📊 === SegNeXt Serving Performance Report ===
💾 Model Size on Disk: 562.50 MB
⏱️ Inference Latency (median): 775.14 ms
⏱️ Inference Latency (95th percentile): 786.92 ms
⏱️ Inference Latency (99th percentile): 794.41 ms
🚀 Inference Throughput (single sample): 1.30 FPS
🚀 Batch Throughput: 1.31 FPS


In [20]:
import torch
from mmseg.apis import init_model
from mmengine.config import Config
import os

# Paths to config and checkpoint
config_path = 'configs/segnext/segnext_mscan-l_1xb16-adamw-40k_plantseg115-512x512.py'
checkpoint_path = 'work_dirs/segnext_mscan-l_1xb16-adamw-40k_plantseg115-512x512/iter_8000.pth'

# Define ONNX output path in the same directory as the checkpoint
checkpoint_dir = os.path.dirname(checkpoint_path)
onnx_output_path = os.path.join(checkpoint_dir, 'segnext_plantseg_iter8000.onnx')

# Load model config and weights
cfg = Config.fromfile(config_path)
cfg.model.pretrained = None
model = init_model(cfg, checkpoint_path, device='cuda')
model.eval()

print(f"Loaded trained model from: {checkpoint_path}")

# Create dummy input of shape 512x512
dummy_input = torch.randn(1, 3, 512, 512, device='cuda')

# Export to ONNX
torch.onnx.export(
    model,
    dummy_input,
    onnx_output_path,
    export_params=True,
    opset_version=11,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={
        'input': {0: 'batch_size'},
        'output': {0: 'batch_size'}
    }
)

# Confirm export
if os.path.exists(onnx_output_path):
    print(f"ONNX model successfully saved to: {onnx_output_path}")
else:
    print("Failed to save ONNX model.")


Loads checkpoint by local backend from path: work_dirs/segnext_mscan-l_1xb16-adamw-40k_plantseg115-512x512/iter_8000.pth
✅ Loaded trained model from: work_dirs/segnext_mscan-l_1xb16-adamw-40k_plantseg115-512x512/iter_8000.pth
✅ ONNX model successfully saved to: work_dirs/segnext_mscan-l_1xb16-adamw-40k_plantseg115-512x512/segnext_plantseg_iter8000.onnx


In [5]:
import torch

print("Torch CUDA available:", torch.cuda.is_available())
print("Torch CUDA version:", torch.version.cuda)
print("cuDNN version:", torch.backends.cudnn.version())
print("Device name:", torch.cuda.get_device_name(0))
import onnxruntime as ort
print("ONNXRuntime version:", ort.__version__)
print("Available providers:", ort.get_available_providers())

Torch CUDA available: True
Torch CUDA version: 11.8
cuDNN version: 8700
Device name: Quadro RTX 6000


In [4]:
import os
import time
import torch
import numpy as np
import onnxruntime as ort
from tqdm import tqdm
from sklearn.metrics import accuracy_score, confusion_matrix
import cv2
from glob import glob

# Path to the ONNX model
onnx_path = 'work_dirs/segnext_mscan-l_1xb16-adamw-40k_plantseg115-512x512/segnext_plantseg_iter8000.onnx'
onnx_model_size = os.path.getsize(onnx_path) / (1024 * 1024)

# Create ONNXRuntime session (use GPU if available)
session = ort.InferenceSession(onnx_path, providers=['CUDAExecutionProvider'])
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

# Image preprocessing (follows MMSeg mean/std normalization)
def preprocess(img):  # img: np.ndarray, shape (H, W, 3), BGR format
    img = cv2.resize(img, (512, 512))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)
    mean = np.array([123.675, 116.28, 103.53])
    std = np.array([58.395, 57.12, 57.375])
    img = (img - mean) / std
    img = np.transpose(img, (2, 0, 1))  # HWC -> CHW
    img = np.expand_dims(img, axis=0)  # CHW -> NCHW
    return img.astype(np.float32)

# Load dataset
img_dir = 'data/plantseg115/images/test'
gt_dir  = 'data/plantseg115/annotations/test'
img_paths = sorted(glob(f"{img_dir}/*.jpg"))
gt_paths  = sorted(glob(f"{gt_dir}/*.png"))

assert len(img_paths) == len(gt_paths) and len(img_paths) > 0, "No image-label pairs found!"

# Run accuracy and mIoU evaluation
all_preds = []
all_labels = []

for img_path, gt_path in tqdm(zip(img_paths, gt_paths), total=len(img_paths)):
    img = preprocess(cv2.imread(img_path))  # shape (1, 3, 512, 512)
    gt  = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)
    gt  = cv2.resize(gt, (512, 512), interpolation=cv2.INTER_NEAREST)

    pred_logits = session.run([output_name], {input_name: img})[0]  # shape: (1, 116, 64, 64)
    pred_mask = np.argmax(pred_logits[0], axis=0)                   # shape: (64, 64)

    # Upsample prediction to 512x512
    pred_mask = cv2.resize(pred_mask.astype(np.uint8), (512, 512), interpolation=cv2.INTER_NEAREST)

    valid = gt != 255
    all_preds.extend(pred_mask[valid].flatten())
    all_labels.extend(gt[valid].flatten())

# Compute accuracy and mIoU
acc = accuracy_score(all_labels, all_preds)
conf = confusion_matrix(all_labels, all_preds, labels=range(116))
intersection = np.diag(conf)
union = conf.sum(1) + conf.sum(0) - intersection
miou = (intersection / np.maximum(union, 1)).mean()

# Measure latency for single-image inference
latencies = []
dummy_img = preprocess(cv2.imread(img_paths[0]))

for _ in range(100):
    start = time.time()
    _ = session.run(None, {input_name: dummy_img})
    latencies.append(time.time() - start)

# Measure throughput for batched inference
batch_input = np.repeat(dummy_img, repeats=4, axis=0)  # batch size = 4
batch_times = []
for _ in range(50):
    start = time.time()
    _ = session.run(None, {input_name: batch_input})
    batch_times.append(time.time() - start)

# Print evaluation summary
print("\n=== ONNX Model Evaluation Report ===")
print(f"Model Size on Disk: {onnx_model_size:.2f} MB")
print(f"Pixel Accuracy: {acc * 100:.2f}%")
print(f"mIoU: {miou * 100:.2f}%")
print(f"Inference Latency (median): {np.median(latencies)*1000:.2f} ms")
print(f"Inference Latency (95th percentile): {np.percentile(latencies, 95)*1000:.2f} ms")
print(f"Throughput (single): {1 / np.mean(latencies):.2f} FPS")
print(f"Batch Throughput (batch size = 4): {4 * 50 / np.sum(batch_times):.2f} FPS")


2025-05-13 02:08:32.673561248 [W:onnxruntime:, session_state.cc:1030 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2025-05-13 02:08:32.673582553 [W:onnxruntime:, session_state.cc:1032 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.
100%|██████████| 2295/2295 [05:57<00:00,  6.42it/s]



📊 === ONNX Model Evaluation Report ===
💾 Model Size on Disk: 187.20 MB
🎯 Pixel Accuracy: 85.60%
📏 mIoU: 27.28%
⏱️  Inference Latency (median): 39.22 ms
⏱️  Inference Latency (95%): 45.77 ms
🚀  Throughput (single): 25.03 FPS
🚀  Batch Throughput (BS=4): 7.07 FPS


In [5]:
import neural_compressor

In [18]:
import os
import cv2
import numpy as np
from glob import glob
import onnx
from neural_compressor import PostTrainingQuantConfig, quantization, metric
from neural_compressor.config import AccuracyCriterion
from neural_compressor.data import Datasets, DataLoader
from neural_compressor.model.onnx_model import ONNXModel

# Path configuration
fp32_onnx_path = "work_dirs/segnext_mscan-l_1xb16-adamw-40k_plantseg115-512x512/segnext_plantseg_iter8000.onnx"
int8_onnx_path = "work_dirs/segnext_mscan-l_1xb16-adamw-40k_plantseg115-512x512/segnext_plantseg_quant_static.onnx"
img_root = "data/plantseg115/images/test"

# Dynamically retrieve ONNX input name
onnx_model = onnx.load(fp32_onnx_path)
input_name = onnx_model.graph.input[0].name  # Usually 'input', but retrieved dynamically for safety

# Load the model
model = ONNXModel(fp32_onnx_path)

# Build the dataloader
dataset = Datasets("onnxruntime")["ImageFolder"](root=img_root)
dataloader = DataLoader(framework="onnxruntime", dataset=dataset)

# Prepare a valid example input
img_path = sorted(glob(f"{img_root}/*.jpg"))[0]
img = cv2.imread(img_path)
img = cv2.resize(img, (512, 512))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = img.astype(np.float32)
mean = np.array([123.675, 116.28, 103.53])
std = np.array([58.395, 57.12, 57.375])
img = (img - mean) / std
img = np.transpose(img, (2, 0, 1))
img = np.expand_dims(img, axis=0)
example_input = {input_name: img.astype(np.float32)}

# Configure quantization parameters (enable smooth quant)
config = PostTrainingQuantConfig(
    approach="static",
    device="cpu",
    quant_format="QOperator",
    calibration_sampling_size=10,
    accuracy_criterion=AccuracyCriterion(criterion="relative", tolerable_loss=0.8),
    example_inputs=example_input,
    recipes={"smooth_quant": True}
)

# Perform quantization
q_model = quantization.fit(
    model=model,
    conf=config,
    calib_dataloader=dataloader,
    eval_dataloader=dataloader,
    eval_metric=metric.Metric(name="topk", k=1)
)

# Save the quantized model
if q_model is not None:
    q_model.save_model_to_file(int8_onnx_path)
    print(f"INT8 ONNX model saved to: {int8_onnx_path}")
else:
    print("Quantization failed: q_model is None. Please check logs and parameter settings.")


2025-05-13 02:45:55 [INFO] Start auto tuning.
2025-05-13 02:45:55 [INFO] Create evaluation function according to evaluation dataloader and metric                and Execute the tuning process.
2025-05-13 02:45:55 [INFO] Adaptor has 5 recipes.
2025-05-13 02:45:55 [INFO] 1 recipes specified by user.
2025-05-13 02:45:55 [INFO] 3 recipes require future tuning.
2025-05-13 02:45:55 [INFO] *** Initialize auto tuning
2025-05-13 02:45:55 [INFO] {
2025-05-13 02:45:55 [INFO]     'PostTrainingQuantConfig': {
2025-05-13 02:45:55 [INFO]         'AccuracyCriterion': {
2025-05-13 02:45:55 [INFO]             'criterion': 'relative',
2025-05-13 02:45:55 [INFO]             'higher_is_better': True,
2025-05-13 02:45:55 [INFO]             'tolerable_loss': 0.8,
2025-05-13 02:45:55 [INFO]             'absolute': None,
2025-05-13 02:45:55 [INFO]             'keys': <bound method AccuracyCriterion.keys of <neural_compressor.config.AccuracyCriterion object at 0x7b18f9a90040>>,
2025-05-13 02:45:55 [INFO]       

❌ 量化失败：q_model 是 None，请检查日志和参数设置。


In [19]:
import os
from neural_compressor import PostTrainingQuantConfig, quantization
from neural_compressor.model.onnx_model import ONNXModel

# File paths
fp32_onnx_path = "work_dirs/segnext_mscan-l_1xb16-adamw-40k_plantseg115-512x512/segnext_plantseg_iter8000.onnx"
int8_onnx_path = "work_dirs/segnext_mscan-l_1xb16-adamw-40k_plantseg115-512x512/segnext_plantseg_quant_dynamic.onnx"

# Load the original ONNX model
model = ONNXModel(fp32_onnx_path)

# Configure dynamic quantization
config = PostTrainingQuantConfig(
    approach="dynamic",           # Use dynamic quantization
    device="cpu",                 # Typically run on CPU
    quant_format="QOperator"      # Use standard quantization format
)

# Perform quantization
q_model = quantization.fit(
    model=model,
    conf=config
)

# Save the quantized INT8 model
if q_model is not None:
    q_model.save_model_to_file(int8_onnx_path)
    print(f"Dynamic quantized model saved to: {int8_onnx_path}")
else:
    print("Quantization failed: q_model is None. Please check the path or model structure.")


2025-05-13 02:47:04 [INFO] Start auto tuning.
2025-05-13 02:47:04 [INFO] Quantize model without tuning!
2025-05-13 02:47:04 [INFO] Quantize the model with default configuration without evaluating the model.                To perform the tuning process, please either provide an eval_func or provide an                    eval_dataloader an eval_metric.
2025-05-13 02:47:04 [INFO] Adaptor has 5 recipes.
2025-05-13 02:47:04 [INFO] 0 recipes specified by user.
2025-05-13 02:47:04 [INFO] 3 recipes require future tuning.
2025-05-13 02:47:04 [INFO] *** Initialize auto tuning
2025-05-13 02:47:04 [INFO] {
2025-05-13 02:47:04 [INFO]     'PostTrainingQuantConfig': {
2025-05-13 02:47:04 [INFO]         'AccuracyCriterion': {
2025-05-13 02:47:04 [INFO]             'criterion': 'relative',
2025-05-13 02:47:04 [INFO]             'higher_is_better': True,
2025-05-13 02:47:04 [INFO]             'tolerable_loss': 0.01,
2025-05-13 02:47:04 [INFO]             'absolute': None,
2025-05-13 02:47:04 [INFO]     

✅ Dynamic quantized model saved to: work_dirs/segnext_mscan-l_1xb16-adamw-40k_plantseg115-512x512/segnext_plantseg_quant_dynamic.onnx


In [2]:
import os
import time
import numpy as np
import onnxruntime as ort
from tqdm import tqdm
from sklearn.metrics import accuracy_score, confusion_matrix
import cv2
from glob import glob

# Path to the quantized ONNX model
onnx_path = 'work_dirs/segnext_mscan-l_1xb16-adamw-40k_plantseg115-512x512/segnext_plantseg_quant_dynamic.onnx'
onnx_model_size = os.path.getsize(onnx_path) / (1024 * 1024)

# Create ONNX Runtime session
session = ort.InferenceSession(onnx_path, providers=['CPUExecutionProvider'])
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

# Image preprocessing (same as training)
def preprocess(img):
    img = cv2.resize(img, (512, 512))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)
    mean = np.array([123.675, 116.28, 103.53])
    std = np.array([58.395, 57.12, 57.375])
    img = (img - mean) / std
    img = np.transpose(img, (2, 0, 1))  # HWC -> CHW
    img = np.expand_dims(img, axis=0)  # CHW -> NCHW
    return img.astype(np.float32)

# Dataset paths
img_dir = 'data/plantseg115/images/test'
gt_dir  = 'data/plantseg115/annotations/test'
img_paths = sorted(glob(f"{img_dir}/*.jpg"))
gt_paths  = sorted(glob(f"{gt_dir}/*.png"))
img_paths = img_paths[:50]
gt_paths = gt_paths[:50]
assert len(img_paths) == len(gt_paths) and len(img_paths) > 0, "No test images found!"

# Evaluate accuracy and mIoU
all_preds, all_labels = [], []

for img_path, gt_path in tqdm(zip(img_paths, gt_paths), total=len(img_paths)):
    img = preprocess(cv2.imread(img_path))
    gt  = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)
    gt  = cv2.resize(gt, (512, 512), interpolation=cv2.INTER_NEAREST)

    pred_logits = session.run([output_name], {input_name: img})[0]  # (1, C, H, W)
    pred_mask = np.argmax(pred_logits[0], axis=0)                   # (H, W)
    pred_mask = cv2.resize(pred_mask.astype(np.uint8), (512, 512), interpolation=cv2.INTER_NEAREST)

    valid = gt != 255
    all_preds.extend(pred_mask[valid].flatten())
    all_labels.extend(gt[valid].flatten())

acc = accuracy_score(all_labels, all_preds)
conf = confusion_matrix(all_labels, all_preds, labels=range(116))
intersection = np.diag(conf)
union = conf.sum(1) + conf.sum(0) - intersection
miou = (intersection / np.maximum(union, 1)).mean()

# Measure inference latency (single image)
latencies = []
dummy_img = preprocess(cv2.imread(img_paths[0]))

for _ in range(100):
    start = time.time()
    _ = session.run(None, {input_name: dummy_img})
    latencies.append(time.time() - start)

# Measure throughput (batch size = 4)
batch_input = np.repeat(dummy_img, repeats=4, axis=0)
batch_times = []
for _ in range(50):
    start = time.time()
    _ = session.run(None, {input_name: batch_input})
    batch_times.append(time.time() - start)

# Print evaluation report
print("\n=== ONNX Quantized Model Evaluation Report ===")
print(f"Model Size on Disk: {onnx_model_size:.2f} MB")
print(f"Pixel Accuracy: {acc * 100:.2f}%")
print(f"mIoU: {miou * 100:.2f}%")
print(f"Inference Latency (median): {np.median(latencies)*1000:.2f} ms")
print(f"Inference Latency (95th percentile): {np.percentile(latencies, 95)*1000:.2f} ms")
print(f"Throughput (single): {1 / np.mean(latencies):.2f} FPS")
print(f"Batch Throughput (batch size = 4): {4 * 50 / np.sum(batch_times):.2f} FPS")


100%|██████████| 50/50 [01:38<00:00,  1.97s/it]



📊 === ONNX Quantized Model Evaluation Report ===
💾 Model Size on Disk: 49.67 MB
🎯 Pixel Accuracy: 81.88%
📏 mIoU: 0.88%
⏱️  Inference Latency (median): 1872.40 ms
⏱️  Inference Latency (95%): 1898.49 ms
🚀  Throughput (single): 0.53 FPS
🚀  Batch Throughput (BS=4): 0.55 FPS


In [2]:
print(session.get_providers())


['CPUExecutionProvider']
